In [1]:
import tensorflow as tf
import numpy as np
%matplotlib inline
import matplotlib.pyplot as plt
from keras.layers import *

Using TensorFlow backend.


In [3]:
input_tensor = Input((400, 80, 1))
x = input_tensor
for i in range(3):
    x = Conv2D(32 * 2 ** i, (3, 3), kernel_initializer='he_normal')(x)
    x = BatchNormalization()(x)
    x = Activation('relu')(x)
    x = Conv2D(32 * 2 ** i, (3, 3), kernel_initializer='he_normal')(x)
    x = BatchNormalization()(x)
    x = Activation('relu')(x)
    x = MaxPooling2D(pool_size=(2, 2))(x)

In [61]:
x.get_shape().as_list()

[None, 1, 2]

In [2]:
irs = tf.InteractiveSession()

In [59]:
a = np.random.randn(10, 10, 10)
a.shape

(10, 10, 10)

In [60]:
shape = tf.squeeze(a).get_shape()

In [43]:
shape[0].value

In [28]:
tf.reshape(a, shape=[shape[0], shape[2] * shape[1]])

<tf.Tensor 'Reshape_2:0' shape=(10, 100) dtype=float64>

In [62]:
x = tf.placeholder(tf.float32, shape=[None, 1, 2], name='x')
shape = x.get_shape()
shape

TensorShape([Dimension(None), Dimension(1), Dimension(2)])

In [65]:
tf.reshape(x, shape=[-1, shape[1] * shape[2]])

<tf.Tensor 'Reshape_9:0' shape=(?, 2) dtype=float32>

In [66]:
tf.shape(x)

<tf.Tensor 'Shape:0' shape=(3,) dtype=int32>

In [69]:
a = [1,2,3]

In [70]:
tf.identity(a)

<tf.Tensor 'Identity_1:0' shape=(3,) dtype=int32>

In [35]:
tf.reshape(x, [-1, 2])

<tf.Tensor 'Reshape_7:0' shape=(?, 2) dtype=float32>

In [31]:
tf.reshape(x, [shape[0], shape[1] * shape[2]])

TypeError: Failed to convert object of type <class 'list'> to Tensor. Contents: [Dimension(None), Dimension(2)]. Consider casting elements to a supported type.

In [72]:
import tensorflow as tf
import numpy as np

In [3]:
# 转化一个序列列表为稀疏矩阵
def sparse_tuple_from(sequences, dtype=np.int32):
    """
    Create a sparse representention of x.
    Args:
        sequences: a list of lists of type dtype where each element is a sequence
    Returns:
        A tuple with (indices, values, shape)
    """
    indices = []
    values = []

    for n, seq in enumerate(sequences):
        # print(n, len(seq))
        indices.extend(zip([n] * len(seq), range(len(seq))))
        values.extend(seq)

    indices = np.asarray(indices, dtype=np.int64)
    values = np.asarray(values, dtype=dtype)
    shape = np.asarray([len(sequences), np.asarray(indices).max(0)[1] + 1], dtype=np.int64)

    return indices, values, shape

In [4]:
DIGITS = '0123456789'
def decode_sparse_tensor(sparse_tensor):
    decoded_indexes = list()
    current_i = 0
    current_seq = []
    for offset, i_and_index in enumerate(sparse_tensor[0]):
        i = i_and_index[0]
        if i != current_i:
            decoded_indexes.append(current_seq)
            current_i = i
            current_seq = list()
        current_seq.append(offset)
    decoded_indexes.append(current_seq)
    result = []
    for index in decoded_indexes:
        result.append(decode_a_seq(index, sparse_tensor))
    return result

def decode_a_seq(indexes, spars_tensor):
    decoded = []
    for m in indexes:
        str = DIGITS[spars_tensor[1][m]]
        decoded.append(str)
    return decoded

In [4]:
irs = tf.InteractiveSession()

In [5]:
s = tf.sparse_placeholder(tf.int32)

In [6]:
s0 = sparse_tuple_from([[1,2,3],[0,1],[3,3,4,5]])
s1 = sparse_tuple_from([[1,1,4],[1,0,1],[3,3,6,5]])
s2 = sparse_tuple_from([[1,0,1]])

In [7]:
S0 = tf.SparseTensor(indices=s0[0], values=s0[1], dense_shape=s0[2])
S1 = tf.SparseTensor(indices=s1[0], values=s1[1], dense_shape=s1[2])
S2 = tf.SparseTensor(indices=s2[0], values=s2[1], dense_shape=s2[2])
S0

In [13]:
edit_dist = tf.edit_distance(S1, S2).eval()
edit_dist

array([0.6666667,       inf,       inf], dtype=float32)

In [9]:
reduce = tf.reduce_mean(edit_dist)  # (0.33333334 + 0.33333334 + 0.25) / 3
reduce.eval(), 1 - reduce.eval()

(inf, -inf)

concat the indices

In [43]:
S2_indices = tf.concat([tf.expand_dims(S2.indices[:, 0] + S1.shape[0], axis=-1), 
           tf.expand_dims(S2.indices[:, 1], axis=-1)], axis=1)
# S2.indices.eval()
# S2.indices.shape

In [44]:
concat_indices = tf.concat([S1.indices, S2_indices], axis=0)
S1.indices.eval().shape, S2.indices.eval().shape, concat_indices.eval().shape

((10, 2), (3, 2), (13, 2))

concat the values

In [9]:
concat_values = tf.concat([S1.values, S2.values], axis=0)
S1.values.eval().shape, S1.values.eval().shape, concat_values.eval().shape

((10,), (10,), (13,))

calculate the dense shape

In [13]:
first_dim = S1.shape[0] + S2.shape[0]
print(S1.shape, S2.shape)
second_dim = max(S1.shape[1], S2.shape[1])
first_dim, second_dim

(3, 4) (1, 3)


(Dimension(4), Dimension(4))

In [45]:
# second_dim = tf.placeholder(dtype=tf.int32)
ultimate = tf.SparseTensor(indices=concat_indices, values=concat_values, dense_shape=[first_dim, second_dim])
ultimate.shape

TensorShape([Dimension(4), Dimension(4)])

In [46]:
ultimate.eval()

SparseTensorValue(indices=array([[0, 0],
       [0, 1],
       [0, 2],
       [1, 0],
       [1, 1],
       [1, 2],
       [2, 0],
       [2, 1],
       [2, 2],
       [2, 3],
       [3, 0],
       [3, 1],
       [3, 2]]), values=array([1, 1, 4, 1, 0, 1, 3, 3, 6, 5, 1, 0, 1], dtype=int32), dense_shape=array([4, 4]))

In [16]:
ss = sparse_tuple_from([[1,1,4],[1,0,1],[3,3,6,5],[1,0,1]])
ss

(array([[0, 0],
        [0, 1],
        [0, 2],
        [1, 0],
        [1, 1],
        [1, 2],
        [2, 0],
        [2, 1],
        [2, 2],
        [2, 3],
        [3, 0],
        [3, 1],
        [3, 2]]),
 array([1, 1, 4, 1, 0, 1, 3, 3, 6, 5, 1, 0, 1], dtype=int32),
 array([4, 4]))

In [53]:
decodeds = [S1, S2]
concat_indices = None
concat_values = None
accelerate_indices = 0
for i in range(len(decodeds)):
    decoded = decodeds[i]
    
    if i == 0:
        concat_indices = decoded.indices
        concat_values = decoded.values
    else:
        decoded_indices = tf.concat([tf.expand_dims(decoded.indices[:, 0] + accelerate_indices, axis=-1),
                                    tf.expand_dims(decoded.indices[:, 1], axis=-1)], axis=1)
        concat_indices = tf.concat([concat_indices, decoded_indices], axis=0)
        concat_values = tf.concat([concat_values, decoded.values], axis=0)

    accelerate_indices += decoded.shape[0]

In [54]:
concat_indices.eval()

array([[0, 0],
       [0, 1],
       [0, 2],
       [1, 0],
       [1, 1],
       [1, 2],
       [2, 0],
       [2, 1],
       [2, 2],
       [2, 3],
       [3, 0],
       [3, 1],
       [3, 2]])

In [70]:
tf.edit_distance(ultimate, ultimate).eval()

array([0., 0., 0., 0.], dtype=float32)

FLAGS测试

In [71]:
tf.app.flags.DEFINE_integer('a', 128, "Rnn Units")

In [85]:
tf.flags.FLAGS['a']

#### tfdebugger

In [92]:
import notebook.nbextensions
import urllib.request
import zipfile
SOURCE_URL = 'https://github.com/ericjang/tdb/releases/download/tdb_ext_v0.1/tdb_ext.zip'

In [ ]:
urllib.request.urlretrieve(SOURCE_URL, 'tdb_ext.zip')

In [ ]:
with zipfile.ZipFile('tdb_ext.zip', "r") as z:
    z.extractall("")
notebook.nbextensions.install_nbextension('tdb_ext',user=True)

In [65]:
input_shape=[None, 400, 80, 1]
X = tf.placeholder(tf.float32, shape=input_shape, name='x')
print(X.shape)
x = tf.nn.max_pool(x, [1, 2, 2, 1], strides=[1, 2, 2, 1], padding='VALID', name='cnn2%s' % i)
print(x.shape)

(?, 400, 80, 1)
(?, 100, 20, 32)
